# Read comments carefully and make the appropriate changes to lines that have been marked by CHANGE.

In [ ]:
import os
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import pytesseract
import time

# CHANGE: Set up tesseract command, this will be different depending on where tesseract is in your device
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'
# CHANGE

# CHANGE: Path to the directory containing PDFs
pdf_directory = "../Comments Received After Field Review" # Adjust this path to which folder you want to convert PDFs from
output_directory = "./Text Files 2"  # Adjust this path to where you want to save the text files
# CHANGE

# NO NEED TO CHANGE
# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)
# NO NEED TO CHANGE

# CHANGE: How many files do you want to convert? I've set it to 200 files
file_count = 0
max_files = 200
# CHANGE

# NO NEED TO CHANGE
# Function to check if PDF is text-based using fitz
def is_text_based(pdf_path):
    try:
        with fitz.open(pdf_path) as doc:
            text = ""
            for page in doc:
                text += page.get_text()
            return bool(text.strip())
    except Exception as e:
        return False

# Start measuring the runtime
start_time = time.time()

# Iterate through PDF files and subdirectories
for root, _, files in os.walk(pdf_directory):
    for file in files:
        if file.lower().endswith('.pdf'):
            pdf_path = os.path.join(root, file)
            
            # Stop processing if the limit is reached
            if file_count >= max_files:
                break

            if is_text_based(pdf_path):
                # Extract text using fitz
                text = ""
                with fitz.open(pdf_path) as doc:
                    for page in doc:
                        text += page.get_text()
                output_file_path = os.path.join(output_directory, f"{os.path.splitext(file)[0]}.txt")
            else:
                # Extract text using pytesseract (OCR for image-based PDF)
                images = convert_from_path(pdf_path)
                text = ""
                for image in images:
                    text += pytesseract.image_to_string(image)
                output_file_path = os.path.join(output_directory, f"image_{os.path.splitext(file)[0]}.txt")
            
            # Save the extracted text
            try:
                with open(output_file_path, 'w', encoding='utf-8') as f:
                    f.write(text)
                
                # Increment file count
                file_count += 1
            except FileNotFoundError as e:
                print(f"Error saving file {output_file_path}: {e}")

# Calculate and print the runtime
end_time = time.time()
runtime = end_time - start_time
print(f"Total runtime: {runtime:.2f} seconds")
#NO NEED TO CHANGE